# Spatial Analysis and Map Visualizations

This analysis is for the graffiti removal request type.  Five visualizations are used to tell the story:
  1.  Dataframe Statistics
  2.  Choropleth map showing annual 311 counts for LA
  3.  Density choropleth
  4.  Static heat map - high density neighborhood
  5.  Animated heat map - high density neighborhood
  6.  Annimated heat map showing daily 311 calls for LA
  
This nb is intended to show the ideas.  For starters I'm just using graffiti reports for 2021.  NC's are based on the certified NC dataset.

In [ ]:
%run start.py

import folium
from folium import plugins
from utils import read_new311_shape, dt_to_object, read_ncs

Two data sets provide the foundation data:

  1. The graffiti subset of the cleaned 311 data
  2. The cleaned version of the certified neighborhood councils

I'm using the functions developed in other notebooks to read the data.  I've also added some date abstractions for upstream processing/querying.

In [ ]:
%%time
graffiti_gdf = read_new311_shape('../data/311/graffiti.geojson.zip')

In [ ]:
graffiti_gdf['dayofyear'] = graffiti_gdf['created_dt'].apply(lambda dt: dt.dayofyear)
graffiti_gdf['day_name'] = graffiti_gdf['created_dt'].apply(lambda dt: dt.day_name())
graffiti_gdf['day'] = graffiti_gdf['created_dt'].apply(lambda dt: dt.date().strftime("%m-%d (%A)"))
graffiti_gdf['month'] = graffiti_gdf['created_dt'].apply(lambda dt: dt.month)

In [ ]:
neighborhoods_gdf = read_ncs()

graffiti_counts = graffiti_gdf['nc'].value_counts().to_frame().reset_index().rename(columns={'index': 'nc_id', 'nc': 'count'})

graffiti_merged_gdf = pd.merge(neighborhoods_gdf, graffiti_counts, how="left", on=["nc_id"])

At this point we have the two baseline data frames that will be used in the following steps.

# 1 - Bar Chart

We'll start with a very basic understanding of the 311 dataframe.  This simple graphic shows annual count / NC.  For simplicity I'm only showing the top 25.

In [ ]:
graffiti_gdf['nc_name'].value_counts()[:25].plot.barh(figsize=(8, 8));

# 2 - Choropleth for LA
 
The Oxford Dictionary defines a choropleth map as 

`a map that uses differences in shading, coloring, or the placing of symbols within predefined areas to indicate the average values of a property or quantity in those areas.`

Specifics of the model:

  1. The coloring scheme ranges from Yellow -> Orange -> Red
  2. Annual reported 311 reports (based on created date) are grouped by Neighborhood Council
  3. These counts are then binned into 6 groups and mapped on the color line
  4. Interpret as higher counts are "brighter" red, lower counts are yellow
  5. This visualiztion supports high level analysis (City Council, empowerla, Service Regions, ...)

The receipe for the choropleth - https://vverde.github.io/blob/interactivechoropleth.html

Note:
Data is a combination of counts from the 311 observations and the NC polygons.

In [ ]:
choropleth_map = folium.Map(location = [34.05, -118.25], zoom_start = 10)
plugins.Fullscreen().add_to(choropleth_map)
folium.Choropleth(
 geo_data=graffiti_merged_gdf,
 name='Choropleth',
 data=graffiti_merged_gdf,
 columns=['nc_id','count'],
 key_on="feature.properties.nc_id",
 fill_color='YlOrRd',
 #threshold_scale=myscale,
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='311 request counts',
 smooth_factor=0
).add_to(choropleth_map)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
nc_info = folium.features.GeoJson(
    graffiti_merged_gdf,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name', 'nc_id', 'count'],
        aliases=['Neighborhood: ','NC ID: ','311 count: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
choropleth_map.add_child(nc_info)
choropleth_map.keep_in_front(nc_info)
folium.LayerControl().add_to(choropleth_map)


choropleth_map

Explore the Neighborhood Councils on the choropleth.  Hover over the darkest read NC.  That is the one we'll explore in a bit more detail.

# 3 - Density Choropleth

So I'm curious about normalizing the Graffiti Removal counts for the choropleth.

Some thoughts:

  1. The OCB might be interested in distribution of raw counts?
  2. Residents may be interested in count per area?
  
Let's run with these thoughts and add a choropleth showing density (count/sq KM) in each NC.  Probably beating a dead horse but what the hey!

**Note:** This section is a bit messy.  I'm using code snippets from the population density notebook and reusing the choropleth code from above.  This should be refactored.

In [ ]:
from pyproj import Geod

geod = Geod(ellps="WGS84")

def square_miles(geo):
    square_meters = abs(geod.geometry_area_perimeter(geo)[0])
    return (square_meters * 10.764) / 27878000

In [ ]:
graffiti_merged_gdf['sq_miles'] = graffiti_merged_gdf.apply(lambda row: square_miles(row.geometry), axis=1)

In [ ]:
graffiti_merged_gdf['density'] = graffiti_merged_gdf.apply(lambda row: row['count'] / row['sq_miles'], axis=1)

I just added a density column to the gdf.  We can use that for a new choropleth.  The results are interesting if not predictable.

In [ ]:
density_choropleth_map = folium.Map(location = [34.05, -118.25], zoom_start = 10)
plugins.Fullscreen().add_to(density_choropleth_map)
folium.Choropleth(
 geo_data=graffiti_merged_gdf,
 name='Choropleth Density',
 data=graffiti_merged_gdf,
 columns=['nc_id','density'],
 key_on="feature.properties.nc_id",
 fill_color='YlOrRd',
 #threshold_scale=myscale,
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='311 request density',
 smooth_factor=0
).add_to(density_choropleth_map)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
density_nc_info = folium.features.GeoJson(
    graffiti_merged_gdf,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name', 'nc_id', 'density'],
        aliases=['Neighborhood: ','NC ID: ','311 density: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
density_choropleth_map.add_child(density_nc_info)
density_choropleth_map.keep_in_front(density_nc_info)
folium.LayerControl().add_to(density_choropleth_map)


density_choropleth_map

What I want to do is compare the two choropleths.  I could scroll up and down, break out the two output cells or ...

Why don't we use two output widgets to look at them side by side!

In [ ]:
count_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

In [ ]:
density_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

In [ ]:
with count_output:
    display(choropleth_map)

In [ ]:
with density_output:
    display(density_choropleth_map)

In [ ]:
HBox([count_output, density_output])

That's enough.  You tell me if this helps the story.

# 4 - Static Heat Map for South Central

The definition I'll use for heatmaps comes from wikipedia - `A heat map is a data visualization technique that shows magnitude of a phenomenon as color.`  More specifically, in our case we're using the concept of geospatial heatmaps, `measure counts by color from green -> red so brighter red represents a greater density of 311 reports.`  First we'll look in 2-D.

Processing steps for this visualization:

1. To simplify I will just use the last quarter (2021) data
2. Use the nc id (78) from the visual inspection above to select 311 observations
3. Use the neighborhood polygon to center the map
4. Build and display the heat map

In [ ]:
south_central_gdf = graffiti_gdf.query(f"nc == 78", engine='python').reset_index().drop(columns=['index'])

south_central_quarter_gdf = south_central_gdf.query(f"month > 9", engine='python').reset_index().drop(columns=['index'])

In [ ]:
south_central_poly_geometry = neighborhoods_gdf.query(f"nc_id == 78", engine='python').reset_index().iloc[0]['geometry']

map_center = [south_central_poly_geometry.centroid.y, south_central_poly_geometry.centroid.x]

In [ ]:
static_hmap = folium.Map(location = map_center, tiles='Stamen Toner', zoom_start = 15)
plugins.Fullscreen().add_to(static_hmap)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in south_central_quarter_gdf.geometry ]
plugins.HeatMap(heat_data).add_to(static_hmap)
static_hmap

Looking at these intensities, I see a pattern.  Do you?

# 5 - Animated Heat Map for South Central

For this viusalization I want to "see" the 311 calls in the static heatmap over time.  Folium as a plugin to do just that.  I found a useful receipe here - https://stackoverflow.com/questions/64325958/heatmapwithtime-plugin-in-folium. 



  1. The data for the visualization uses the same data set for the static heatmap
  2. The time component is when the 311 report was created
  3. I do a simple mapping to day-of-the-year for the created_dt
  4. The animation steps through the observations by day-of-the-year
  5. The animation control is placed bottom left in the map display

In [ ]:
south_central_hmap = folium.Map(location = map_center, tiles='Stamen Toner', zoom_start = 14)
plugins.Fullscreen().add_to(south_central_hmap);

In [ ]:
from collections import defaultdict, OrderedDict

south_central_data = defaultdict(list)
for r in south_central_quarter_gdf.itertuples():
    south_central_data[r.day].append([r.latitude, r.longitude])
    
south_central_data = OrderedDict(sorted(south_central_data.items(), key=lambda t: t[0]))

In [ ]:
south_central_hm = plugins.HeatMapWithTime(data=list(south_central_data.values()),
                     index=list(south_central_data.keys()), 
                     radius=10,
                     auto_play=True,
                     max_opacity=0.8)

In [ ]:
south_central_hm.add_to(south_central_hmap)
south_central_hmap

Look at the animation in full screen mode.  You can see that most reports seem to be on weekdays?  That may warrent further investigation at some point.

# 6 - Animated Heat Map for LA

Now I return to the whole 311 data set for graffiti removal.  We started with the choropleth for all of LA, did an excursion to the high density neighborhood, and now we'll go back to the entire city for the entire year.

  1. The data for the visualization uses 311 data for the graffiti
  2. The time component is when the 311 report was created
  3. I do a simple mapping to day-of-the-year for the created_dt
  4. The animation steps through the observations by day-of-the-year
  5. The animation control is placed bottom left in the map display



This receipe helped.  https://stackoverflow.com/questions/64325958/heatmapwithtime-plugin-in-folium

Note:  This is covering the whole city so it's a bit hard to get a good starting zoom level.  Once the animation starts you should go to full screen mode and the navigate around zoom in (with the mouse scoll).  That's a good way to see what's happening.

In [ ]:
hmap = folium.Map(location = [34.05, -118.25], tiles='Stamen Toner', zoom_start = 10)
plugins.Fullscreen().add_to(hmap);

In [ ]:
from collections import defaultdict, OrderedDict

data = defaultdict(list)
for r in graffiti_gdf.itertuples():
    data[r.day].append([r.latitude, r.longitude])
    
data = OrderedDict(sorted(data.items(), key=lambda t: t[0]))

In [ ]:
hm = plugins.HeatMapWithTime(data=list(data.values()),
                     index=list(data.keys()), 
                     radius=10,
                     auto_play=True,
                     max_opacity=0.8)

In [ ]:
hm.add_to(hmap)
hmap

# Summary

I've created a basic story flow going from a large scale choropleth map to small scale, area specific maps, and finally back to the large scale animation.  This should give you some sense for what happened in 2021 for the graffiti removal request type.  It also highlights some other areas for exploration.